In [1]:
import pandas as pd
import numpy as np
import oracledb
from sqlalchemy import create_engine
import os
import glob
pd.set_option('display.max_columns',100)
import time
import seaborn as sns
import matplotlib.pyplot as plt

- Perfiorming: 0 DPD
- Grace Period: 1-29 DPD
- Delinquents: Accounts with >=30 days DPD

3 Approaches has been done:
1. Bulk query everything as one: does not work, processing too slow in VDI, still lacking important data -> XP
2. Separate queries for multiple tables: does not work during data loading because of memory error
3. Separate queries for multiple tables but direct upload to BQ (success)

# A. Query Data

## 1. DBA Connection

In [2]:
# credentials
usernameDBA = os.environ['UID']
passwordDBA = os.environ['DBAPW']
dsnDBA = '10.220.50.121:1661/ANALYTIC'

In [3]:
# create connection
connectionDBA = oracledb.connect(user=usernameDBA, password=passwordDBA, dsn=dsnDBA)

In [4]:
# create engine
engineDBA = create_engine('oracle+oracledb://', creator=lambda: connectionDBA)

## 2. EDWB Connection

In [5]:
# credentials
usernameEDWB = os.environ['UID']
passwordEDWB = os.environ['EDWB_PW']
dsnEDWB = '172.16.2.28:1521/brgprod'

In [6]:
# create connection
connectionEDWB = oracledb.connect(user=usernameEDWB, password=passwordEDWB, dsn=dsnEDWB)

In [7]:
# create engine
engineEDWB = create_engine('oracle+oracledb://', creator=lambda: connectionEDWB)

## 3. Run Query

In [8]:
# create list of all dates
allDates = ['20241231','20241129','20241031','20240930','20240830','20240731','20240628',
            '20240531','20240430','20240328','20240229','20240131','20231229']
# no dpd for <202404
# ['20241129','20241031','20240930','20240829','20240731','20240628','20240531',
#             '20240430','20240328','20240229','20240131','20231229','20231130']

In [ ]:
# # open sql file Master Lending
# with open("CCPull_ML.sql", 'r') as file:
#     query_ML = file.read()

### a. Customer Portfolio

In [10]:
# open sql file Customer Portfolio
with open("CCPull_CP.sql", 'r') as file:
    query_CP = file.read()
    
# RUN CP
CPCollector = []
for date in allDates:
    yearmonth = date[2:6]
    dateAlt = date[2:]
    if date=='20240628': # for a case where 20240628 data is not available
        try:
            # print in case of errors
            print(f'Running CP Query 20240628')

            # open and read the file as a single buffer
            sqlQuery = query_CP.format(dt='20240626', ym='2406')

            # run query and store data in df
            print('CP run')
            with engineDBA.connect() as connection:
                CP = pd.read_sql(sqlQuery, connection) 
            print(f'CP 20240628 queried successfully')

            # change column names into UPPER CASE
            CP.columns = [x.upper() for x in CP.columns]
        except Exception as e:
            print(e)
            CP = pd.DataFrame()
        finally:
            CPCollector.append(CP)

    else:
        try:
            # print in case of errors
            print(f'Running CP Query {date}')

            # open and read the file as a single buffer
            sqlQuery = query_CP.format(dt=date, ym=yearmonth)

            # run query and store data in df
            print('CP run')
            with engineDBA.connect() as connection:
                CP = pd.read_sql(sqlQuery, connection) 
            print(f'CP {date} queried successfully')

            # change column names into UPPER CASE
            CP.columns = [x.upper() for x in CP.columns]
        except Exception as e:
            print(e)
            CP = pd.DataFrame()
        finally:
            CPCollector.append(CP)


Running CP Query 20241231
CP run
CP 20241231 queried successfully
Running CP Query 20241129
CP run
CP 20241129 queried successfully
Running CP Query 20241031
CP run
CP 20241031 queried successfully
Running CP Query 20240930
CP run
CP 20240930 queried successfully
Running CP Query 20240829
CP run
CP 20240829 queried successfully
Running CP Query 20240731
CP run
CP 20240731 queried successfully
Running CP Query 20240628
CP run
CP 20240628 queried successfully
Running CP Query 20240531
CP run
CP 20240531 queried successfully
Running CP Query 20240430
CP run
CP 20240430 queried successfully
Running CP Query 20240328
CP run
CP 20240328 queried successfully
Running CP Query 20240229
CP run
CP 20240229 queried successfully
Running CP Query 20240131
CP run
CP 20240131 queried successfully
Running CP Query 20231229
CP run
CP 20231229 queried successfully


In [11]:
# concat all data into one
CPAll = pd.concat(CPCollector, ignore_index=True)

In [12]:
# view dataframe info
CPAll.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2092325 entries, 0 to 2092324
Data columns (total 41 columns):
 #   Column            Dtype         
---  ------            -----         
 0   BASE_DT           object        
 1   GCIF_NO           object        
 2   SEGMENT           object        
 3   CUST_NM           object        
 4   OPEN_DATE         datetime64[ns]
 5   LAST_ACTIVE_DATE  datetime64[ns]
 6   CUST_TP           object        
 7   GENDER_CD         object        
 8   AGE               float64       
 9   SAGA_FLAG         object        
 10  EMPLOYMENT_TYPE   object        
 11  INCOME_TYPE       object        
 12  MARITAL_STATUS    object        
 13  MICRO_SEGMENT     object        
 14  FUNDING_NOA       int64         
 15  FUNDING_BAL       float64       
 16  FUNDING_AVG       float64       
 17  NOA_DORMANT       int64         
 18  ALL_DORMANT       int64         
 19  CC                int64         
 20  PLOAN             int64         
 21  TR      

In [ ]:
# check base_dt
CPAll['BASE_DT'].value_counts().sort_index()

BASE_DT
20231229    160700
20240131    160446
20240229    160200
20240328    159729
20240430    159325
20240531    159349
20240628    159699
20240731    160612
20240830    161202
20240930    162203
20241031    162990
20241129    163161
20241231    162709
Name: count, dtype: int64

In [15]:
# name masking function
def masking_name(param):
    if param not in ["nan","NaN"]:
        return  ' '.join([item.replace(item[1:-1], "*"*len(item[1:-1])) if len(item)>2 and item[-1]!="," and item[-1]!="." else (item.replace(item[1:-2], "*"*len(item[1:-2])) if len(item)>2 and (item[-1]==",") else (item if len(item)>2 and (item[-1]==".") else item.replace(item[1:-2], "*"*len(item[1:-2])))) for item in param.split()])
    else:
        return param
CPAll['CUST_NM'] = CPAll['CUST_NM'].astype(str).apply(masking_name) 

In [16]:
CPAll

,BASE_DT,GCIF_NO,SEGMENT,CUST_NM,OPEN_DATE,LAST_ACTIVE_DATE,CUST_TP,GENDER_CD,AGE,SAGA_FLAG,EMPLOYMENT_TYPE,INCOME_TYPE,MARITAL_STATUS,MICRO_SEGMENT,FUNDING_NOA,FUNDING_BAL,FUNDING_AVG,NOA_DORMANT,ALL_DORMANT,CC,PLOAN,TR,PB,PRK,PPB,BG,LC,KPR,KPM,WOM,COLLECT,MF,BONDS,AUM,LUM,TRB,M2U_USER,CRTRX_MTD,CRTRX_AMT_MTD,DBTRX_MTD,DBTRX_AMT_MTD
0,20241231,G000005421,None,H***G L*M L**,1999-05-05,NaT,INDV,M,65.0,SAGA,OTHERS,NON-FIXED INCOME,MARRIED,5.Golden Age,0,0.000000e+00,0.000000e+00,0,0,1,0,0,0,0,0,0,0,0,0,0,1.0,0,0,0.000000e+00,29482651.00,2.948265e+07,0,0,0.0,0,0.0
1,20241231,G000040295,CFS-RB-5MB,H*****A L*******I,2009-12-11,2024-12-23,INDV,F,53.0,SAGA,EMPLOYED PERMANENT,FIXED INCOME,MARRIED,4.Silver Age,2,7.183330e+05,3.382327e+05,0,0,1,0,0,0,0,0,0,0,0,0,0,1.0,0,0,7.183330e+05,40181082.00,4.089942e+07,1,4,2950001.0,3,2214366.0
2,20241231,G000041632,CFS-RB-3EA,Y******N,2000-03-22,2024-12-31,INDV,M,61.0,SAGA,RETIRED,NON-FIXED INCOME,WIDOWED,5.Golden Age,7,7.830976e+06,7.779952e+06,3,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0,0,7.830976e+06,0.00,7.830976e+06,0,2,702982.0,2,100596.0
3,20241231,G000047647,None,S T*****A S*****I,2012-09-28,NaT,INDV,M,63.0,SAGA,OTHERS,NON-FIXED INCOME,MARRIED,5.Golden Age,0,0.000000e+00,0.000000e+00,0,0,1,0,0,0,0,0,0,0,0,0,0,1.0,0,0,0.000000e+00,0.00,0.000000e+00,0,0,0.0,0,0.0
4,20241231,G000080747,None,A S*****A P*****O,2002-03-11,NaT,INDV,M,53.0,SAGA,OTHERS,NON-FIXED INCOME,MARRIED,4.Silver Age,0,0.000000e+00,0.000000e+00,0,0,1,0,0,0,0,0,0,0,0,0,0,1.0,0,0,0.000000e+00,1412311.00,1.412311e+06,0,0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2092320,20231229,G710124645,CFS-RB-1PW,DR H S*I S*****I,2023-08-25,2023-10-10,INDV,F,67.0,SAGA,EMPLOYED PERMANENT,FIXED INCOME,WIDOWED,5.Golden Age,1,1.168001e+09,1.165893e+09,0,0,1,0,0,0,0,0,0,0,0,0,0,1.0,0,0,1.168001e+09,2645837.00,1.170647e+09,0,0,0.0,2,3267462.0
2092321,20231229,G104945217,CFS-RB-5MB,A*E K****A,2011-07-12,2023-12-27,INDV,M,36.0,NON-SAGA,EMPLOYED PERMANENT,FIXED INCOME,MARRIED,3.Mid Lifers,1,3.142194e+06,5.456744e+05,0,0,1,1,0,0,0,0,0,0,0,0,0,1.0,0,0,3.142194e+06,8306296.00,1.144849e+07,1,1,43397389.0,11,40525056.0
2092322,20231229,G509874103,None,D****L O***O,2022-08-01,NaT,INDV,M,32.0,NON-SAGA,SELF EMPLOYED,NON-FIXED INCOME,MARRIED,2.Young Families,0,0.000000e+00,0.000000e+00,0,0,1,0,0,0,0,0,0,0,0,1,0,1.0,0,0,0.000000e+00,74753180.17,7.475318e+07,0,0,0.0,0,0.0
2092323,20231229,G303167472,None,E**Y S*****I H***P,2008-08-19,NaT,INDV,F,52.0,SAGA,HOUSEWIFE,NON-FIXED INCOME,MARRIED,4.Silver Age,0,0.000000e+00,0.000000e+00,0,0,1,0,0,0,0,0,0,0,0,0,0,1.0,0,0,0.000000e+00,29028425.00,2.902842e+07,0,0,0.0,0,0.0


In [ ]:
# save data
CPAll.to_csv(f'D:/Work/SAGA CC Bad Debt/Data/CP/CP_All_2312-2412v2.csv')

### b. Master Lending Summary (GCIF Level)

In [11]:
# open sql file Master Lending grouped
with open("CCPull_ML_GPDv2.sql", 'r') as file:
    query_ML_SUM = file.read()

# RUN ML Sum
MLSumCollector = []
for date in allDates:
    yearmonth = date[2:6]
    dateAlt = date[2:]
    # Query ML
    if date=='20240628':
        try:
            '20240626'
            # print in case of errors
            print(f'Running ML Sum Query 20240626')

            # open and read the file as a single buffer
            sqlQuery = query_ML_SUM.format(dt='20240626', ym=yearmonth)

            # run query and store data in df
            print('ML Sum run')
            with engineDBA.connect() as connection:
                MLSum = pd.read_sql(sqlQuery, connection) 
            print(f'ML Sum {date} queried successfully')

            # change column names into UPPER CASE
            MLSum.columns = [x.upper() for x in MLSum.columns]
        except Exception as e:
            print(e)
            MLSum = pd.DataFrame()
        finally:
           MLSumCollector.append(MLSum)
    else:
        try:
            # print in case of errors
            print(f'Running ML Query {date}')

            # open and read the file as a single buffer
            sqlQuery = query_ML_SUM.format(dt=date, ym=yearmonth)

            # run query and store data in df
            print('ML Sum run')
            with engineDBA.connect() as connection:
                MLSum = pd.read_sql(sqlQuery, connection) 
            print(f'ML Sum {date} queried successfully')

            # change column names into UPPER CASE
            MLSum.columns = [x.upper() for x in MLSum.columns]
        except Exception as e:
            print(e)
            MLSum = pd.DataFrame()
        finally:
           MLSumCollector.append(MLSum)

Running ML Query 20241231
ML Sum run


ML Sum 20241231 queried successfully
Running ML Query 20241129
ML Sum run
ML Sum 20241129 queried successfully
Running ML Query 20241031
ML Sum run
ML Sum 20241031 queried successfully
Running ML Query 20240930
ML Sum run
ML Sum 20240930 queried successfully
Running ML Query 20240830
ML Sum run
ML Sum 20240830 queried successfully
Running ML Query 20240731
ML Sum run
ML Sum 20240731 queried successfully
Running ML Sum Query 20240626
ML Sum run
ML Sum 20240628 queried successfully
Running ML Query 20240531
ML Sum run
ML Sum 20240531 queried successfully
Running ML Query 20240430
ML Sum run
ML Sum 20240430 queried successfully
Running ML Query 20240328
ML Sum run
ML Sum 20240328 queried successfully
Running ML Query 20240229
ML Sum run
ML Sum 20240229 queried successfully
Running ML Query 20240131
ML Sum run
ML Sum 20240131 queried successfully
Running ML Query 20231229
ML Sum run
ML Sum 20231229 queried successfully


In [12]:
# concat all data into one
MLSumAll = pd.concat(MLSumCollector, ignore_index=True)

In [13]:
# view dataframe info
MLSumAll.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2771277 entries, 0 to 2771276
Data columns (total 18 columns):
 #   Column             Dtype  
---  ------             -----  
 0   BASE_DT            object 
 1   APPL_ID            object 
 2   GCIF_NO            object 
 3   NOA                int64  
 4   ORG_LMT            int64  
 5   CURR_LMT           float64
 6   CURR_OS            int64  
 7   MAX_OS             int64  
 8   AVG_OS             float64
 9   INT_AMT            float64
 10  CP_CLCT            object 
 11  RESTRUCT_1         float64
 12  RESTRUCT_5         object 
 13  RESTRUCT_8         float64
 14  PASTDUE_DAYS_CALC  float64
 15  PASTDUE_DT_MAX     object 
 16  PASTDUE_DAYS       int64  
 17  CP_STATUS          object 
dtypes: float64(6), int64(5), object(7)
memory usage: 380.6+ MB


In [14]:
# check base_dt
MLSumAll[['BASE_DT','APPL_ID']].value_counts().sort_index()

BASE_DT   APPL_ID
20231229  CP         161565
          XP          51501
20240131  CP         161307
          XP          51654
20240229  CP         161053
          XP          51518
20240328  CP         160707
          XP          51512
20240430  CP         160368
          XP          51218
20240531  CP         160164
          XP          51230
20240626  CP         160510
          XP          51340
20240731  CP         161408
          XP          51259
20240830  CP         162189
          XP          51325
20240930  CP         162991
          XP          51401
20241031  CP         163716
          XP          51328
20241129  CP         163875
          XP          51239
20241231  CP         163523
          XP          51376
Name: count, dtype: int64

In [15]:
MLSumAll

,BASE_DT,APPL_ID,GCIF_NO,NOA,ORG_LMT,CURR_LMT,CURR_OS,MAX_OS,AVG_OS,INT_AMT,CP_CLCT,RESTRUCT_1,RESTRUCT_5,RESTRUCT_8,PASTDUE_DAYS_CALC,PASTDUE_DT_MAX,PASTDUE_DAYS,CP_STATUS
0,20241231,CP,G308872746,1,3000000,4.900000e+07,10000,3619425,1.813559e+06,2.100000e+03,00001,1.0,None,NaN,0.0,00000000,0,PERFORMING
1,20241231,XP,G802495028,21,5500000000,2.765500e+08,276549998,313183331,2.944487e+08,0.000000e+00,00001,NaN,None,NaN,NaN,None,0,PERFORMING
2,20241231,CP,G605894112,2,64000000,1.228000e+07,40000,40000,2.423077e+04,8.400000e+03,00001,2.0,None,NaN,0.0,00000000,0,PERFORMING
3,20241231,CP,G804728756,6,660000000,1.151837e+08,47061597,135551577,8.719481e+07,9.882935e+06,00001,6.0,None,NaN,0.0,00000000,0,PERFORMING
4,20241231,CP,G002435485,7,924000000,4.074135e+07,22262730,48275479,2.179952e+07,4.675173e+06,00001,7.0,None,NaN,0.0,00000000,0,PERFORMING
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2771272,20231229,XP,G300470531,3,2700000000,1.129915e+08,112991547,112991547,1.129915e+08,1.021444e+06,00001,NaN,None,NaN,NaN,None,0,PERFORMING
2771273,20231229,CP,G809695430,4,279189200,2.357777e+08,246159329,246159329,2.461593e+08,5.169346e+07,00004,3.0,None,1.0,148.0,20231204,120,DELINQUENT
2771274,20231229,CP,G608411559,2,173488400,1.509724e+08,157712123,157712123,1.577121e+08,3.311955e+07,00005,1.0,None,1.0,129.0,20230822,210,DELINQUENT
2771275,20231229,CP,G300397031,2,440000000,1.042747e+08,104274676,104274676,1.042747e+08,2.189768e+07,00002,2.0,None,NaN,25.0,20231204,1,GRACE PERIOD


In [16]:
# save data
MLSumAll.to_csv(f'D:/Work/SAGA CC Bad Debt/Data/ML_Sum/ML_Sum_2312-2412V2.csv')

### c. Master Loan (Account Level) [Skip]

In [ ]:
# RUN ML Summary
for date in allDates:
    yearmonth = date[2:6]
    dateAlt = date[2:]
    # Query ML
    try:
        # print in case of errors
        print(f'Running ML Sum Query {date}')

        # open and read the file as a single buffer
        sqlQuery = query_ML_SUM.format(dt=date, ym=yearmonth)

        # run query and store data in df
        print('ML Sum run')
        with engineDBA.connect() as connection:
            MLSum = pd.read_sql(sqlQuery, connection) 
        print(f'ML Sum {date} queried successfully')

        # change column names into UPPER CASE
        MLSum.columns = [x.upper() for x in MLSum.columns]
    except Exception as e:
        print(e)
        MLSum = pd.DataFrame()
    finally:
        MLSum.to_csv(f'D:/Work/SAGA CC Bad Debt/Data/ML_Sum/MLSum_{date}.pkl')

Running ML Sum Query 20241129
ML Sum run
ML Sum 20241129 queried successfully
Running ML Sum Query 20241031
ML Sum run
ML Sum 20241031 queried successfully
Running ML Sum Query 20240930
ML Sum run
ML Sum 20240930 queried successfully
Running ML Sum Query 20240829
ML Sum run
ML Sum 20240829 queried successfully
Running ML Sum Query 20240731
ML Sum run
ML Sum 20240731 queried successfully
Running ML Sum Query 20240628
ML Sum run
ML Sum 20240628 queried successfully
Running ML Sum Query 20240531
ML Sum run
ML Sum 20240531 queried successfully
Running ML Sum Query 20240430
ML Sum run
ML Sum 20240430 queried successfully
Running ML Sum Query 20240328
ML Sum run
ML Sum 20240328 queried successfully
Running ML Sum Query 20240229
ML Sum run
ML Sum 20240229 queried successfully
Running ML Sum Query 20240131
ML Sum run
ML Sum 20240131 queried successfully
Running ML Sum Query 20231229
ML Sum run
ML Sum 20231229 queried successfully
Running ML Sum Query 20231130
ML Sum run
ML Sum 20231130 queried

### d. Delinquency (Payment Ratio)

In [9]:
# open sql file Delq_History 
with open("Delq_Hist.sql", 'r') as file:
    query_DELQ = file.read()

PRCollector = []
# RUN DELQ
for date in allDates:
    yearmonth = date[2:6]
    dateAlt = date[2:]
    # query payment ratio
    if date=='20240829':
        try:
            # print in case of errors
            print(f'Running Delq/Payment Ratio Query {date}')

            # open and read the file as a single buffer
            sqlQuery = query_DELQ.format(dt='240830', ym=yearmonth)

            # run query and store data in df
            print('Delq/Payment Ratio run')
            with engineEDWB.connect() as connection:
                PR = pd.read_sql(sqlQuery, connection) 
            print(f'Delq/Payment Ratio {date} queried successfully')

            # change column names into UPPER CASE
            PR.columns = [x.upper() for x in PR.columns]
        except Exception as e:
                print(e)
                PR = pd.DataFrame()
        finally:
            PRCollector.append(PR)         
    else:
        try:
            # print in case of errors
            print(f'Running Delq/Payment Ratio Query {date}')

            # open and read the file as a single buffer
            sqlQuery = query_DELQ.format(dt=dateAlt, ym=yearmonth)

            # run query and store data in df
            print('Delq/Payment Ratio run')
            with engineEDWB.connect() as connection:
                PR = pd.read_sql(sqlQuery, connection) 
            print(f'Delq/Payment Ratio {date} queried successfully')

            # change column names into UPPER CASE
            PR.columns = [x.upper() for x in PR.columns]
        except Exception as e:
                print(e)
                PR = pd.DataFrame()
        finally:
            PRCollector.append(PR)

Running Delq/Payment Ratio Query 20241231
Delq/Payment Ratio run
Delq/Payment Ratio 20241231 queried successfully
Running Delq/Payment Ratio Query 20241129
Delq/Payment Ratio run
Delq/Payment Ratio 20241129 queried successfully
Running Delq/Payment Ratio Query 20241031
Delq/Payment Ratio run
Delq/Payment Ratio 20241031 queried successfully
Running Delq/Payment Ratio Query 20240930
Delq/Payment Ratio run
Delq/Payment Ratio 20240930 queried successfully
Running Delq/Payment Ratio Query 20240830
Delq/Payment Ratio run
Delq/Payment Ratio 20240830 queried successfully
Running Delq/Payment Ratio Query 20240731
Delq/Payment Ratio run
Delq/Payment Ratio 20240731 queried successfully
Running Delq/Payment Ratio Query 20240628
Delq/Payment Ratio run
Delq/Payment Ratio 20240628 queried successfully
Running Delq/Payment Ratio Query 20240531
Delq/Payment Ratio run
Delq/Payment Ratio 20240531 queried successfully
Running Delq/Payment Ratio Query 20240430
Delq/Payment Ratio run
Delq/Payment Ratio 2024

In [10]:
# concat all data into one
PRAll = pd.concat(PRCollector, ignore_index=True)

In [11]:
# view dataframe info
PRAll.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3715755 entries, 0 to 3715754
Data columns (total 25 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   BASE_DT       int64 
 1   CARD_NMBR     int64 
 2   DELQ_HIST_1   object
 3   DELQ_HIST_2   object
 4   DELQ_HIST_3   object
 5   DELQ_HIST_4   object
 6   DELQ_HIST_5   object
 7   DELQ_HIST_6   object
 8   DELQ_HIST_7   object
 9   DELQ_HIST_8   object
 10  DELQ_HIST_9   object
 11  DELQ_HIST_10  object
 12  DELQ_HIST_12  object
 13  DELQ_HIST_13  object
 14  DELQ_HIST_14  object
 15  DELQ_HIST_15  object
 16  DELQ_HIST_16  object
 17  DELQ_HIST_17  object
 18  DELQ_HIST_18  object
 19  DELQ_HIST_19  object
 20  DELQ_HIST_20  object
 21  DELQ_HIST_21  object
 22  DELQ_HIST_22  object
 23  DELQ_HIST_23  object
 24  DELQ_HIST_24  object
dtypes: int64(2), object(23)
memory usage: 708.7+ MB


In [12]:
# check delq hist
PRAll['BASE_DT'].value_counts().sort_index()

BASE_DT
231229    291437
240131    289328
240229    287632
240328    286647
240430    284732
240531    283255
240628    283995
240731    284951
240830    285907
240930    285560
241031    285260
241129    284331
241231    282720
Name: count, dtype: int64

In [13]:
PRAll

,BASE_DT,CARD_NMBR,DELQ_HIST_1,DELQ_HIST_2,DELQ_HIST_3,DELQ_HIST_4,DELQ_HIST_5,DELQ_HIST_6,DELQ_HIST_7,DELQ_HIST_8,DELQ_HIST_9,DELQ_HIST_10,DELQ_HIST_12,DELQ_HIST_13,DELQ_HIST_14,DELQ_HIST_15,DELQ_HIST_16,DELQ_HIST_17,DELQ_HIST_18,DELQ_HIST_19,DELQ_HIST_20,DELQ_HIST_21,DELQ_HIST_22,DELQ_HIST_23,DELQ_HIST_24
0,241231,4423730000272008,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,241231,4423730000925001,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z
2,241231,4423730001027013,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,0,B,B
3,241231,4423730001369100,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z
4,241231,4423730001584013,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3715750,231229,5155959956193006,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,B,4,3,2
3715751,231229,5155959956341001,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B
3715752,231229,5155959956344013,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3715753,231229,5155959956348006,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z,Z


In [14]:
PRAll['BASE_DT'] = '20'+PRAll['BASE_DT'].astype(str)

In [34]:
PRAll['BASE_DT'].replace('20240829','20240830', inplace=True)

C:\Users\MR38804\AppData\Local\Temp\ipykernel_15368\3294274728.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  PRAll['BASE_DT'].replace('20240829','20240830', inplace=True)


In [36]:
PRAll['BASE_DT'].value_counts().sort_index()

BASE_DT
20231229    291437
20240131    289328
20240229    287632
20240328    286647
20240430    284732
20240531    283255
20240628    283995
20240731    284951
20240830    285907
20240930    285560
20241031    285260
20241129    284331
20241231    282720
Name: count, dtype: int64

In [16]:
# read account mapper
GCIF_ACCT = pd.read_csv('D:/Work/SAGA CC Bad Debt/Data/ACCT_GCIF/DATE_GCIF_NOTEv2.csv')

In [17]:
# change base_dt to string
GCIF_ACCT['BASE_DT'] = GCIF_ACCT['BASE_DT'].astype(str)

In [18]:
# check field info
GCIF_ACCT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9389164 entries, 0 to 9389163
Data columns (total 3 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   BASE_DT  object
 1   GCIF_NO  object
 2   NOTE_NO  int64 
dtypes: int64(1), object(2)
memory usage: 214.9+ MB


In [19]:
GCIF_ACCT.value_counts('BASE_DT')

BASE_DT
20231229    800885
20241231    755708
20241129    748578
20241031    741009
20240930    734777
20240830    725415
20240731    717478
20240628    707600
20240531    700137
20240430    696053
20240328    692098
20240229    687111
20240131    682315
Name: count, dtype: int64

In [20]:
GCIF_ACCT[GCIF_ACCT[['BASE_DT','NOTE_NO']].duplicated(keep=False)].sort_values('NOTE_NO').head(10)

,BASE_DT,GCIF_NO,NOTE_NO


In [21]:
GCIF_ACCT[GCIF_ACCT['NOTE_NO']==3562850001438003].sort_values('BASE_DT')

,BASE_DT,GCIF_NO,NOTE_NO
3083049,20231229,G900369489,3562850001438003
1187779,20240131,G900369489,3562850001438003
4716706,20240229,G900369489,3562850001438003
6780653,20240328,G900369489,3562850001438003
6205470,20240430,G900369489,3562850001438003
8859642,20240531,G903966117,3562850001438003
216017,20240628,G903966117,3562850001438003
3890750,20240731,G903966117,3562850001438003
8452464,20240830,G903966117,3562850001438003
4647184,20240930,G903966117,3562850001438003


In [37]:
PRAllJoined = PRAll.merge(GCIF_ACCT, left_on=['BASE_DT','CARD_NMBR'], right_on=['BASE_DT','NOTE_NO'], how='left')

In [38]:
PRAllJoined.shape

(3715755, 27)

In [39]:
PRAllJoined[PRAllJoined['GCIF_NO'].isnull()].value_counts('BASE_DT')

BASE_DT
20231229    3
20240131    3
20240229    3
20240328    3
20240430    3
20240531    3
20240628    3
20240731    3
20240830    3
20240930    3
20241031    3
20241129    3
20241231    3
Name: count, dtype: int64

In [40]:
PRAllJoined.columns

Index(['BASE_DT', 'CARD_NMBR', 'DELQ_HIST_1', 'DELQ_HIST_2', 'DELQ_HIST_3',
       'DELQ_HIST_4', 'DELQ_HIST_5', 'DELQ_HIST_6', 'DELQ_HIST_7',
       'DELQ_HIST_8', 'DELQ_HIST_9', 'DELQ_HIST_10', 'DELQ_HIST_12',
       'DELQ_HIST_13', 'DELQ_HIST_14', 'DELQ_HIST_15', 'DELQ_HIST_16',
       'DELQ_HIST_17', 'DELQ_HIST_18', 'DELQ_HIST_19', 'DELQ_HIST_20',
       'DELQ_HIST_21', 'DELQ_HIST_22', 'DELQ_HIST_23', 'DELQ_HIST_24',
       'GCIF_NO', 'NOTE_NO'],
      dtype='object')

In [41]:
PRAllJoined = PRAllJoined[['BASE_DT', 'CARD_NMBR','GCIF_NO', 'NOTE_NO', 
        'DELQ_HIST_1', 'DELQ_HIST_2', 'DELQ_HIST_3',
       'DELQ_HIST_4', 'DELQ_HIST_5', 'DELQ_HIST_6', 'DELQ_HIST_7',
       'DELQ_HIST_8', 'DELQ_HIST_9', 'DELQ_HIST_10', 'DELQ_HIST_12',
       'DELQ_HIST_13', 'DELQ_HIST_14', 'DELQ_HIST_15', 'DELQ_HIST_16',
       'DELQ_HIST_17', 'DELQ_HIST_18', 'DELQ_HIST_19', 'DELQ_HIST_20',
       'DELQ_HIST_21', 'DELQ_HIST_22', 'DELQ_HIST_23', 'DELQ_HIST_24',
       ]]

In [42]:
PRAllJoined['DELQ_HIST_1'].value_counts()

DELQ_HIST_1
Z    1630263
B     994294
0     473403
1     472606
2      86287
3      23504
4       7500
5       5577
6       4073
7       3357
9         25
Name: count, dtype: int64

In [43]:
PRAllJoined['INACTIVE'] = PRAllJoined['DELQ_HIST_1'].apply(lambda x: 1 if x=='Z' else None)
PRAllJoined['MINIMUM_PYMT'] = PRAllJoined['DELQ_HIST_1'].apply(lambda x: 1 if x=='1' else None)
PRAllJoined['NOTHING_DUE'] = PRAllJoined['DELQ_HIST_1'].apply(lambda x: 1 if x=='0' else None)
PRAllJoined['FULL_PYMT'] = PRAllJoined['DELQ_HIST_1'].apply(lambda x: 1 if x=='B' else None)
PRAllJoined['GRACE_PERIOD'] = PRAllJoined['DELQ_HIST_1'].apply(lambda x: 1 if x=='2' else None)
PRAllJoined['DELINQUENT'] = PRAllJoined['DELQ_HIST_1'].apply(lambda x: 1 if str(x) in '3456789' else None)

In [44]:
PRAllJoined[PRAllJoined['DELQ_HIST_1']=='3'].head(20)

,BASE_DT,CARD_NMBR,GCIF_NO,NOTE_NO,DELQ_HIST_1,DELQ_HIST_2,DELQ_HIST_3,DELQ_HIST_4,DELQ_HIST_5,DELQ_HIST_6,DELQ_HIST_7,DELQ_HIST_8,DELQ_HIST_9,DELQ_HIST_10,DELQ_HIST_12,DELQ_HIST_13,DELQ_HIST_14,DELQ_HIST_15,DELQ_HIST_16,DELQ_HIST_17,DELQ_HIST_18,DELQ_HIST_19,DELQ_HIST_20,DELQ_HIST_21,DELQ_HIST_22,DELQ_HIST_23,DELQ_HIST_24,INACTIVE,MINIMUM_PYMT,NOTHING_DUE,FULL_PYMT,GRACE_PERIOD,DELINQUENT
303,20241231,4423730054689016,G000445353,4.423730e+15,3,2,0,B,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,1.0
588,20241231,4423730104540003,G000331587,4.423730e+15,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,2,2,2,2,2,NaN,NaN,NaN,NaN,NaN,1.0
688,20241231,4423730125046014,G000433326,4.423730e+15,3,3,3,2,3,3,3,3,3,3,2,2,0,2,2,2,0,2,2,0,2,2,1,NaN,NaN,NaN,NaN,NaN,1.0
1301,20241231,4423730194835008,G206357501,4.423730e+15,3,2,0,4,3,2,0,2,0,2,2,0,2,4,4,4,5,6,5,4,3,2,0,NaN,NaN,NaN,NaN,NaN,1.0
1381,20241231,4423730199415004,G902222611,4.423730e+15,3,2,2,2,2,2,2,1,0,2,1,0,2,2,0,2,1,0,2,0,2,2,2,NaN,NaN,NaN,NaN,NaN,1.0
2801,20241231,4423730338915013,G806315412,4.423730e+15,3,3,3,3,3,3,3,3,3,4,3,3,3,3,3,3,3,3,3,3,3,6,6,NaN,NaN,NaN,NaN,NaN,1.0
3392,20241231,4423740014225025,G200389039,4.423740e+15,3,2,2,3,3,3,3,3,3,3,3,3,3,3,3,3,3,2,2,2,2,2,0,NaN,NaN,NaN,NaN,NaN,1.0
3482,20241231,4423740019466012,G300549284,4.423740e+15,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,NaN,NaN,NaN,NaN,NaN,1.0
3568,20241231,4423740023763016,G700398274,4.423740e+15,3,2,0,Z,Z,Z,Z,Z,B,3,B,B,2,0,0,0,0,0,0,0,0,B,0,NaN,NaN,NaN,NaN,NaN,1.0
3571,20241231,4423740023811013,G400744785,4.423740e+15,3,2,0,Z,Z,Z,Z,Z,Z,Z,Z,B,2,0,Z,Z,Z,Z,Z,Z,Z,Z,Z,NaN,NaN,NaN,NaN,NaN,1.0


In [45]:
PRGPD = PRAllJoined[['BASE_DT', 'GCIF_NO', 'NOTE_NO', 'INACTIVE',
       'MINIMUM_PYMT', 'NOTHING_DUE', 'FULL_PYMT', 'GRACE_PERIOD',
       'DELINQUENT']].groupby(['BASE_DT','GCIF_NO']).sum().reset_index()

In [46]:
PRGPD

,BASE_DT,GCIF_NO,NOTE_NO,INACTIVE,MINIMUM_PYMT,NOTHING_DUE,FULL_PYMT,GRACE_PERIOD,DELINQUENT
0,20231229,EX0000000009531047,4.184883e+16,4.0,0.0,1.0,4.0,0.0,0.0
1,20231229,EX0000000009531054,4.649870e+15,0.0,0.0,0.0,1.0,0.0,0.0
2,20231229,EX0000000009531062,9.217690e+15,1.0,0.0,0.0,1.0,0.0,0.0
3,20231229,EX0000000009531070,9.299740e+15,2.0,0.0,0.0,0.0,0.0,0.0
4,20231229,EX0000000009531088,2.316730e+16,2.0,1.0,2.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
2103494,20241231,G910459114,3.562850e+15,0.0,0.0,0.0,0.0,0.0,0.0
2103495,20241231,G910459239,3.562850e+15,0.0,0.0,0.0,0.0,0.0,0.0
2103496,20241231,G910459247,3.562850e+15,0.0,0.0,0.0,0.0,0.0,0.0
2103497,20241231,G910459858,3.562850e+15,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
# PRAllJoined.to_csv('D:/Work/SAGA CC Bad Debt/Data/PR/PR_GCIF_ALL.csv', index=False)

In [47]:
PRGPD.to_csv('D:/Work/SAGA CC Bad Debt/Data/PR/PR_GCIF_GPDv2.csv', index=False)

### e. Write Off

In [2]:
# reda WO data
WO = pd.read_csv('D:/Work/SAGA CC Bad Debt/Data/CC_WO/CC_WRITEOFF_2412.csv')

In [3]:
WO

,GCIF_NO,AGREE_ID,WRITE_OFF_AMT,WRITE_OFF_DT
0,G800454746,CP0010014423730000298003,0,20121208
1,G300019213,CP0010014423730000565013,2064414,20240924
2,G400348108,CP0010014423730000593015,0,20070119
3,G900563065,CP0010014423730001438012,0,20160321
4,G600362511,CP0010014423730001486011,0,20070927
...,...,...,...,...
30449,G609284807,XP001532515595999747001724081122060,0,20241023
30450,G609284807,XP001532515595999747001724081122061,0,20241023
30451,G609284807,XP001532515595999747001724082125110,0,20241023
30452,G609284807,XP001532515595999747001724082125111,0,20241023


In [9]:
# group WO by GCIF
WO_GPD = WO.groupby(['GCIF_NO']).agg(WO_NOA=('AGREE_ID','count'),
                                     WO_AMT_SUM=('WRITE_OFF_AMT','sum'),
                                     WO_DT_LAST=('WRITE_OFF_DT','max')).reset_index()

In [10]:
WO_GPD

,GCIF_NO,WO_NOA,WO_AMT_SUM,WO_DT_LAST
0,EX0000000009531013,2,385838383,20230824
1,EX0000000009531021,2,1368811,20230210
2,EX0000000009531039,1,596848,20210111
3,EX0000000009531054,4,4035520,20231112
4,EX0000000009531062,2,2842011,20230612
...,...,...,...,...
19478,G910049345,1,427419,20240607
19479,G910118777,1,8013778,20240714
19480,G910140938,2,10446686,20240906
19481,G910147032,6,11369793,20241025


In [13]:
# save CSV
WO_GPD.to_csv('D:/Work/SAGA CC Bad Debt/Data/CC_WO/CC_WO_GPD_2412.csv', index=False)